In [ ]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
#import keras.backend as K
#K.image_dim_ordering='th'


In [ ]:
batch_size = 40
nb_classes = 9
#image_size=(218,303)
#input_shape=(3,218,303)
image_size=(128,128)
input_shape=(3,128,128)
classes=['bird','bluebird','chicken','cobra','finch','flamingo','frog','ostrich','salamander']
cpmap={'n01807496': 'bird', 'n01580077': 'bluebird', 'n01514859': 'chicken',
       'n01748264': 'cobra', 'n01601694': 'finch', 'n02007558': 'flamingo',
       'n01644373': 'frog', 'n01518878': 'ostrich', 'n01692333': 'salamander'}

In [ ]:
# Change this to correct location
imagedir = '/data/cs231n'
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        imagedir + '/train',
        target_size=image_size,
        batch_size=batch_size,
        class_mode='categorical')
val_datagen = ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow_from_directory(
        imagedir + '/val',
        target_size=image_size,
        batch_size=batch_size,
        class_mode='categorical')

print(val_generator.class_indices)
for sub in train_generator.class_indices.keys():
    print('%s ==> %s' % (sub, classes[train_generator.class_indices[sub]]))

In [ ]:
nb_epoch = 50
nb_filters=32
kernel_size=(3,3)
pool_size=(2,2)

In [ ]:
model = Sequential()
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=input_shape))
model.add(Activation('relu'))
print(model.output_shape)
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1]))
model.add(Activation('relu'))
print(model.output_shape)
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
print(model.output_shape)
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
print(model.output_shape)
model.add(Activation('softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

In [ ]:
#model.fit_generator(train_generator,samples_per_epoch=1000, nb_epoch=nb_epoch)
hist = model.fit_generator(train_generator,samples_per_epoch=200, nb_epoch=nb_epoch,
                    validation_data=val_generator, nb_val_samples=50)


In [ ]:
score = model.evaluate_generator(train_generator, val_samples=50)
print('Test score:', score[0])
print('Test accuracy:', score[1])

In [ ]:
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.show()
plt.plot(hist.history['acc'])
plt.plot(hist.history['val_acc'])
plt.show()

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        imagedir + '/test',
        target_size=image_size,
        batch_size=batch_size,
        shuffle=False,
        class_mode=None)
pred = model.predict_generator(test_generator, test_generator.nb_sample)

In [ ]:
#indices = np.argmax(pred, axis=1)
#print(indices)
np.set_printoptions(precision=2, suppress=True)
ci1 = 0
ci2 = 0
ci3 = 0
for (f,ids) in zip(test_generator.filenames, pred):
    idx = np.argsort(ids)[-1:-4:-1]
    pc = classes[idx[0]]
    pref = f[4:13:]
    ac = cpmap[pref]
    if(pc == ac):
        ci1 += 1
    else:
        pc = classes[idx[1]]
        if(pc == ac):
            ci2 += 1
        else:
            pc = classes[idx[2]]
            if(pc == ac):
                ci3 += 1
            else:
                print(pc, ac)
        
print(ci1, ci2, ci3)


In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("first_model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("first_model.h5")
print("Saved model to disk")